In [1]:
import requests
import urllib.request
import numpy as np
import time
from bs4 import BeautifulSoup
import re
import pandas as pd
import pickle
from sklearn.metrics import mean_squared_error, mean_squared_log_error

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)



In [2]:
with open('phillies_2019.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    phillies_2019 = pickle.load(f)

In [3]:
with open('phils_prior.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    phils_prior = pickle.load(f)

In [4]:
with open('prediction.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    prediction = pickle.load(f)

In [5]:
with open('prediction1.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    prediction1 = pickle.load(f)

# Combine and Re-Organize Data

In [6]:
phils_total_sum = pd.concat([phils_prior, phillies_2019])

In [7]:
phils_total_sum.reset_index(drop=True, inplace = True)

Y = phils_total_sum.opp_Win

phils_total_sum = phils_total_sum.drop(['opp_Win'], axis=1)
phils_total_sum = phils_total_sum.drop(['Game', 'opp_Game', 'IP', 'opp_IP'], axis=1)


In [8]:
int_columns = phils_total_sum.columns[:-2]

In [9]:
for x in int_columns:
    phils_total_sum[x] = phils_total_sum[x].astype(str).astype(int)

In [10]:
phils_total_sum.insert(loc=12, column = 'BA', value = phils_total_sum['H']/phils_total_sum['AB'])

In [11]:
phils_total_sum.insert(loc=34, column = 'opp_BA', value = phils_total_sum['opp_H']/phils_total_sum['opp_AB'])

In [12]:
phils_total_sum.insert(loc=13, column = 'OBP', value = (phils_total_sum['H'] + phils_total_sum['BB'] + phils_total_sum['opp_HBP'])/ phils_total_sum['PA'])

In [13]:
phils_total_sum.insert(loc=36, column = 'opp_OBP', value = (phils_total_sum['opp_H'] + phils_total_sum['opp_BB'] + phils_total_sum['HBP'])/ phils_total_sum['opp_PA'])

In [14]:
phils_total_sum.insert(loc=14, column = 'SLG', value = (((phils_total_sum['H']-(phils_total_sum['2B']+phils_total_sum['3B']+phils_total_sum['HR']))+
                                                        (phils_total_sum['2B']*2) + (phils_total_sum['3B'] *3) + (phils_total_sum['HR']*4))/ 
                                                        phils_total_sum['AB']))





In [15]:
phils_total_sum.insert(loc=38, column = 'opp_SLG', value = (((phils_total_sum['opp_H']-(phils_total_sum['opp_2B']+phils_total_sum['opp_3B']+phils_total_sum['opp_HR']))+
                                                        (phils_total_sum['opp_2B']*2) + (phils_total_sum['opp_3B'] *3) + (phils_total_sum['opp_HR']*4))/ 
                                                        phils_total_sum['opp_AB']))





In [16]:
phils_total_sum.insert(loc=15, column = 'OPS', value = phils_total_sum['OBP'] + phils_total_sum['SLG'])




In [17]:
phils_total_sum.insert(loc=40, column = 'opp_OPS', value = phils_total_sum['opp_OBP'] + phils_total_sum['opp_SLG'])




In [18]:
features_team = [col for col in phils_total_sum.columns if phils_total_sum[col].dtype in [np.object]]
X_team = phils_total_sum[features_team]

np.shape(X_team)

(754, 2)

In [19]:
# Make dummies
X_team = pd.get_dummies(X_team, drop_first=True)
X_team = X_team.loc[1:,:]
X_team.reset_index(inplace=True)

# Logistic Regression

In [20]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [21]:
averages = pd.DataFrame(columns = phils_total_sum.columns[:-2])

for i in tqdm(range(1,len(phils_total_sum))):
    if i <=15:
        temp = phils_total_sum.loc[:i, :].mean()
        temp = pd.DataFrame(temp).T
        temp['opp_Win'] = Y[i]
        averages = pd.concat([averages,temp], ignore_index=True)
    elif i > 15:
        temp = phils_total_sum.loc[(i-3):i, :].mean()
        temp = pd.DataFrame(temp).T
        temp['opp_Win'] = Y[i]
        averages = pd.concat([averages,temp], ignore_index=True)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


In [22]:
X = averages.drop(columns = ('opp_Win'))
Y = averages['opp_Win']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0)

In [23]:
# y_train = y_train.astype(int)

from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(fit_intercept = False, C = 1e12, solver='liblinear')
model_log = logreg.fit(X_train, y_train)
# model_log

In [24]:
y_hat_test = logreg.predict(X_test)
y_hat_train = logreg.predict(X_train)

In [25]:
import numpy as np
#We could subtract the two columns. If values or equal, difference will be zero. Then count number of zeros.
residuals = np.abs(y_train - y_hat_train)
print(pd.Series(residuals).value_counts())
print(pd.Series(residuals).value_counts(normalize=True))

0.0    416
1.0    148
Name: opp_Win, dtype: int64
0.0    0.737589
1.0    0.262411
Name: opp_Win, dtype: float64


In [26]:
print('Training r^2:', logreg.score(X_train.astype(int), y_train.astype(int)))
print('Testing r^2:', logreg.score(X_test.astype(int), y_test.astype(int)))
print('Training MSE:', mean_squared_error(y_train.astype(int), logreg.predict(X_train.astype(int))))
print('Testing MSE:', mean_squared_error(y_test.astype(int), logreg.predict(X_test.astype(int))))

Training r^2: 0.475177304964539
Testing r^2: 0.455026455026455
Training MSE: 0.524822695035461
Testing MSE: 0.544973544973545


In [27]:
from sklearn import preprocessing

# scale the data and perform train test split
X_scaled = preprocessing.scale(X)

# X_train, X_test, y_train, y_test = train_test_split(X_scaled,Y)

In [28]:
logreg = LogisticRegression(fit_intercept = False, C = 1e0, solver='liblinear')
model_log = logreg.fit(X_train.astype(int), y_train.astype(int))
print('Training r^2:', logreg.score(X_train.astype(int), y_train.astype(int)))
print('Testing r^2:', logreg.score(X_test.astype(int), y_test.astype(int)))
print('Training MSE:', mean_squared_error(y_train.astype(int), logreg.predict(X_train.astype(int))))
print('Testing MSE:', mean_squared_error(y_test.astype(int), logreg.predict(X_test.astype(int))))

Training r^2: 0.7127659574468085
Testing r^2: 0.6666666666666666
Training MSE: 0.2872340425531915
Testing MSE: 0.3333333333333333


In [29]:
X_all = pd.concat([pd.DataFrame(X_scaled), X_team], axis = 1)
# X_train, X_test, y_train, y_test = train_test_split(X_scaled,Y)
X_train = X_scaled[:600,:]
X_test = X_scaled[600:, :]
y_train = Y.loc[:599]
y_test = Y.loc[600:]

In [30]:
logreg_all = LogisticRegression(fit_intercept = True, C = 1e0, penalty='l1')
logreg_all.fit(X_train.astype(int), y_train.astype(int))
print('Training r^2:', logreg_all.score(X_train.astype(int), y_train.astype(int)))
print('Testing r^2:', logreg_all.score(X_test.astype(int), y_test.astype(int)))
print('Training MSE:', mean_squared_error(y_train.astype(int), logreg_all.predict(X_train.astype(int))))
print('Testing MSE:', mean_squared_error(y_test.astype(int), logreg_all.predict(X_test.astype(int))))

Training r^2: 0.7233333333333334
Testing r^2: 0.6601307189542484
Training MSE: 0.27666666666666667
Testing MSE: 0.33986928104575165


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [31]:
#X_test[-1, :]

In [32]:
# corr = phils_total_sum.corr()
# corr[corr<-0.5]

# Evaluating Logistic Regression

In [33]:
def precision(y_hat, y):
    #Could also use confusion matrix
    y_y_hat = list(zip(y, y_hat))
    tp = sum([1 for i in y_y_hat if i[0]==1 and i[1]==1])
    fp = sum([1 for i in y_y_hat if i[0]==0 and i[1]==1])
    return tp/float(tp+fp)

def recall(y_hat, y):
    #Could also use confusion matrix
    y_y_hat = list(zip(y, y_hat))
    tp = sum([1 for i in y_y_hat if i[0]==1 and i[1]==1])
    fn = sum([1 for i in y_y_hat if i[0]==1 and i[1]==0])
    return tp/float(tp+fn)

def accuracy(y_hat, y):
    #Could also use confusion matrix
    y_y_hat = list(zip(y, y_hat))
    tp = sum([1 for i in y_y_hat if i[0]==1 and i[1]==1])
    tn = sum([1 for i in y_y_hat if i[0]==0 and i[1]==0])
    return (tp+tn)/float(len(y_hat))

def f1(y_hat,y):
    precision_score = precision(y_hat,y)
    recall_score = recall(y_hat,y)
    numerator = precision_score * recall_score
    denominator = precision_score + recall_score
    return 2 * (numerator / denominator)

y_hat_test = logreg.predict(X_test)
y_hat_train = logreg.predict(X_train)

#print('Training Precision: ', precision(y_hat_train, y_train))
#print('Testing Precision: ', precision(y_hat_test, y_test))
#print('\n\n')

#print('Training Recall: ', recall(y_hat_train, y_train))
#print('Testing Recall: ', recall(y_hat_test, y_test))
#print('\n\n')

print('Training Accuracy: ', accuracy(y_hat_train, y_train))
print('Testing Accuracy: ', accuracy(y_hat_test, y_test))
print('\n\n')

#print('Training F1-Score: ',f1(y_hat_train,y_train))
#print('Testing F1-Score: ',f1(y_hat_test,y_test))

Training Accuracy:  0.6216666666666667
Testing Accuracy:  0.6339869281045751





# XG Boost

In [34]:
import xgboost as xgb
import pandas as pd
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import warnings
from sklearn import preprocessing
warnings.filterwarnings('ignore')
%matplotlib inline

In [35]:
#labels = Y
#labels_removed_df = X
#scaler = StandardScaler()
#scaled_df = scaler.fit_transform(labels_removed_df)
X = averages.drop(columns = ('opp_Win'))
X_scaled = preprocessing.scale(X)

# X_all = pd.concat([pd.DataFrame(X_scaled), X_team], axis = 1)
X_train = X_scaled[:600,:]
X_test = X_scaled[600:, :]
y_train = Y.loc[:599]
y_test = Y.loc[600:]


In [36]:
X_train.shape

(600, 50)

In [37]:
def prob_odds(game):
    prob_phils_win = game[1]
    prob_other_win = game[0]
    if prob_phils_win >0.5:
        x = ((-prob_phils_win)*100)/(1-prob_phils_win)
        y = (100-(prob_other_win*100))/prob_other_win
    else:
        x = (100-(prob_phils_win*100))/prob_phils_win
        y = ((-prob_other_win)*100)/(1-prob_other_win)
    return x, y

In [76]:
clf = xgb.XGBClassifier(max_depth = 1, min_child_weight = 4)
clf.fit(X_train, y_train)
training_preds = clf.predict(X_train)
val_preds = clf.predict(X_test)
training_accuracy = accuracy_score(y_train.values.astype(int), training_preds.astype(int))
val_accuracy = accuracy_score(y_test.values.astype(int), val_preds.astype(int))

print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))

Training Accuracy: 73.33%
Validation accuracy: 65.36%


In [216]:
with open('prediction1.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    prediction1 = pickle.load(f)

In [217]:
prediction_proba = clf.predict_proba(prediction1.values)

In [218]:
prediction_proba[0]

array([0.40942627, 0.5905737 ], dtype=float32)

In [219]:
prob_odds(prediction_proba[0])

(-144.24421894610202, 144.24421894610202)

In [220]:
Historical_odds2 = pd.DataFrame(prob_odds(prediction_proba[0])).T

In [221]:
Historical_odds2.rename(columns = {0:'Phillies Odds', 1:'Opponent Odds'}, inplace=True)

In [222]:
Historical_odds2['Date'] = ''
Historical_odds2['Phillies Westgate Odds'] = ''
Historical_odds2['Opponent Westgate Odds'] = ''

In [223]:
Historical_odds2['Date'][0] = 'July 18, 2019'
Historical_odds2['Phillies Westgate Odds'][0] = -103
Historical_odds2['Opponent Westgate Odds'][0] = -107

In [224]:
Historical_odds = pd.concat([Historical_odds, Historical_odds2])

In [226]:
Historical_odds['Opponent'] = ''

In [228]:
Historical_odds.reset_index(inplace=True)

In [237]:
Historical_odds['Opponent'][4] = 'Dodgers'

In [243]:
Historical_odds.drop(['index'], axis=1, inplace=True)

In [247]:
Historical_odds.to_html('index.html')

In [ ]:
import pickle

with open('BG_predictions.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(Historical_odds, f, pickle.HIGHEST_PROTOCOL)
    

In [ ]:
Historical_odds['Phillies Actual Odds'][0] = -130

In [ ]:
Historical_odds['Opponent Actual Odds'][0] = 120

In [ ]:
Historical_odds['3'] = ''

In [ ]:
Historical_odds['4'] = ''

In [ ]:
Historical_odds.rename(columns ={'Phillies Actual Odds': 'Phillies Westgate Odds', 'Opponent Actual Odds': 'Opponent Westgate Odds'}, inplace=True)




In [ ]:
Historical_odds.reset_index(inplace=True)


In [ ]:
clf.feature_importances_[38]

In [ ]:
X.columns[38]

In [ ]:
game = clf.predict_proba(X_test[0,:].reshape(1,-1)).tolist()

In [ ]:
(100 - game[0][0]*100) / game[0][0]

In [ ]:
for x in range(12,20):
    game = clf.predict_proba(X_test[-x,:].reshape(1,-1)).tolist()[0]
    print(prob_odds(game))